# Hosting models on Grid

Grid offers both: Machine Learning as a Service and Encrypted Machine Learning as a service. In this series of tutorials we show how you can serve and query models on Grid.

## 3. Host and query an encrypted model

In the previous tutorial we served a CNN for classifying images with different 2 types of skin deseases: benign keratosis and melanoma (type of skin cancer). In this tutorial we show how to serve this model on a **encrypted way** on Grid.

### Imports and model specifications

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr
import helper

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

# Connect nodes to each other
gr.connect_all_nodes([bob, alice, charlie, dan])

### Load dataset

In [2]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:484: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


### Making a model ready to be served and encrypted

In order to serve the model it needs to be serializable. A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. You can learn more about plans in [Syft's tutorials](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2008%20-%20Introduction%20to%20Plans.ipynb).


### Define Model

Let's load the model we just trained.

In [3]:
model = helper.make_model(is_plan=True)
# model.load_state_dict(th.load("binary-skin-cancer-detection-model"))

In [ ]:
model.build(data)

In [ ]:
helper.test(model, test_generator)

Average loss: 0.6926, Accuracy: 565/1106 (51.1%)



(0.6925812635240676, 51.08499095840868)

## Serve model

In [ ]:
alice.serve_model(model, is_private_model=True)

In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr
import helper

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

# Connect nodes to each other
gr.connect_all_nodes([bob, alice, charlie, dan])

### Load dataset

In [2]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

/home/marianne/PySyft/syft/frameworks/torch/hook/hook.py:484: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  current_tensor = hook_self.torch.native_tensor(*args, **kwargs)


In [3]:
x_ptr = data.fix_precision().share(bob, charlie, crypto_provider=dan)

# TODO: this should be stored automatically
me._objects[x_ptr.id] = x_ptr

### Get a copy of the private model

In [4]:
private_model = alice.get_private_model(model_owner=me, location=alice,
                                        model_id="convnet", workers=[bob, charlie],
                                        crypto_provider=dan)
print(private_model)

<Plan Net id:70975214434 owner:me built>


### Run encrypted inference

In [ ]:
%%time
print(private_model(x_ptr).get().float_prec())

In [ ]:
model._objects

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

## Star PySyft on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

[Star PySyft](https://github.com/OpenMined/PySyft)

## Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

## Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

[PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
[Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

## Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)